In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Risvandiani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w) 
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

113 documents
30 classes ['Identity', 'activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'inspire', 'insult', 'jokes', 'karan', 'news', 'nicetty', 'no', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']
119 unique stemmed words ["'m", "'s", ',', '10', '19', 'a', 'ag', 'am', 'anyon', 'ar', 'ask', 'awesom', 'bad', 'bby', 'be', 'best', 'bye', 'can', 'contact', 'could', 'covid', 'cre', 'cricket', 'cur', 'dat', 'day', 'design', 'develop', 'do', 'doing', 'dumb', 'fin', 'for', 'funny', 'get', 'good', 'goodby', 'googl', 'gre', 'hah', 'he', 'hello', 'help', 'hey', 'hi', 'hol', 'hot', 'how', 'i', 'idiot', 'ind', 'inspir', 'internet', 'is', 'it', 'jok', 'kar', 'know', 'lat', 'latest', 'laugh', 'lmao', 'lol', 'lost', 'mad', 'mak', 'malik', 'match', 'me', 'mot', 'namast', 'new', 'next', 'nic', 'no', 'nop', 'off', 'ok', 'old', 'program', 'provid', 'quest', 'riddl'

In [5]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
        
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[: ,0])
train_y = list(training[: ,1])

In [6]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 14999  | total loss: 0.01348 | time: 0.113s
| Adam | epoch: 1000 | loss: 0.01348 - acc: 0.9824 -- iter: 112/113
Training Step: 15000  | total loss: 0.01216 | time: 0.121s
| Adam | epoch: 1000 | loss: 0.01216 - acc: 0.9842 -- iter: 113/113
--
INFO:tensorflow:C:\Users\Risvandiani\Kecerdasan Buatan (AI)\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [22]:
import pickle
pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open("training_data","wb"))

In [23]:
data = pickle.load( open( "training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [24]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Risvandiani\Kecerdasan Buatan (AI)\model.tflearn


In [25]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
        return(np.array(bag))

In [30]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)

In [31]:
print("Press 0 if you don't want to chat with our chatbot.")
while True:
    message = input("")
    if message == "0":
        break
    result = response(message)
    print(result)

Press 0 if you don't want to chat with our chatbot.
None
Glad I could make you laugh !
None
yeah
Yeah!
None
hello
Hello
None
0
